In [42]:
import numpy as np
import matplotlib.image as matlib
import sys
import os, os.path

In [43]:
histology = "/Users/peterschmidt/brainatlas/images/P57-16/histology"
mri_indices = "/Users/peterschmidt/brainatlas/images/P57-16/mri/indices_axial"
mri_matrices = "/Users/peterschmidt/brainatlas/images/P57-16/mri/matrices"
slice_dir = "slices_HE"
matrix = "block_"

In [44]:
class Histology:
    
    def __init__(self):
        self._images = 0
        self.x = 0
        self.y = 0
        self.z = 0
        self.matrix = None
        
    def parseBlockFolder(self, block_name):
        folder = os.path.join(histology, block_name)
        slice_folder = os.path.join(folder,"slices_HE")
        if not os.path.exists(slice_folder):
            print("we couldn't find {}".format(slice_folder))
            return
        images = [name for name in os.listdir(slice_folder) if os.path.isfile(os.path.join(slice_folder, name))]
        count = len(images)
#        print("found {} images in folder {}".format(count, slice_folder))
        self._images = count
        if count > 0:
            firstImage = images[0]
            fullpath = os.path.join(slice_folder, firstImage)
            fullimage = matlib.imread(fullpath)
            self.x,self.y,rgb = fullimage.shape
#            print("x = {}, y = {}, rgb = {}".format(x,y,rgb))
        
        
        try:
            self.z = int(block_name)
#            print(self._z)
        except ValueError:
            self.z = 0
        
        self._getTransformationMatrix(block_name)
        
    def _getTransformationMatrix(self, block_name):
        if block_name == "" or block_name == "0":
            return
        matrix_name = matrix + block_name + ".txt"
        filepath = os.path.join(mri_matrices,matrix_name)
        self.matrix = np.loadtxt(filepath, delimiter=" ")
 #       print(self._matrix)
        
        

In [45]:
histBlock = Histology()
histBlock.parseBlockFolder("25")

In [46]:
def collect_histology_data():
    block_list = []
    blocks = [name for name in os.listdir(histology) if os.path.isdir(os.path.join(histology, name))]
    for block in blocks:
        hist_block = Histology()
        hist_block.parseBlockFolder(block)
        block_list.append(hist_block)
    return block_list



In [52]:
hist_blocks = collect_histology_data()

In [55]:
def checkConsistency(block, vector):
    if block >= len(hist_blocks):
        print("there is an inconsistency between a found block ID {} and the number of blocks {} in histology".format(block, len(hist_blocks)))
        return false
    
    histo_data = hist_blocks[block]
    trafo = histo_data.matrix
    if trafo == None:
        print("We do not have a valid transformation matrix")
        return false
    
    hist_coordinates = trafo.dot(vector)
    x = math.floor(hist_coordinates[0])
    y = math.floor(hist_coordinates[1])
    z = math.floor(hist_coordinates[2])
    
    if histo_data.x >= x and histo_data.y >= y and histo_data.z >= z:
        return true
    else:
        print("[x,y,z] are inconsistent: x_h={} x={}, y_h={}, y={}, z_h={}, z={}".format(histo_data.x,x,histo_data.y,y,histo_data.z,z))
        return false


def parseNPYFiles():
    axial_slices = [name for name in os.listdir(mri_indices) if os.path.isfile(os.path.join(mri_indices, name))]
    z = 0
    for slice in axial_slices:
        current_block = 0
        found_block = 0
        pathname = os.path.join(mri_indices, slice)
        numpyarray = np.load(pathname)
        for x in numpyarray:
            for y in x:
                current_block = numpyarray[x,y]
                if np.any(current_block > 0) and found_block != current_block:
                    found_block = current_block
                    vector = np.array([x,y,z])
                    checkConsistency(current_block, vector)
        z = z + 1
    

    

In [56]:
parseNPYFiles()